# N-Grams

In [1]:
import logging
from ekorpkit import eKonf

logging.basicConfig(level=logging.WARNING)
print(eKonf.__version__)

0.1.31+19.gdb9b114.dirty


## Tokenize and extract tokens

In [2]:
corpus_cfg = eKonf.compose(config_group="corpus")
corpus_cfg.name = "bok_minutes"
corpus_cfg.data_dir = "/workspace/data/datasets/corpus/ekorpkit"


In [3]:
tkn_cfg = eKonf.compose(config_group="preprocessor/tokenizer=mecab")
cfg = eKonf.compose(config_group="pipeline")
cfg.data.corpus = corpus_cfg
cfg._pipeline_ = ["tokenize", "explode_splits", "save_dataframe"]
cfg.num_workers = 100
cfg.tokenize.preprocessor.tokenizer = tkn_cfg
cfg.explode_splits.id_key = "id"
cfg.explode_splits.split_key = "sent_id"
cfg.explode_splits.separator = "\n"
cfg.save_dataframe.output_dir = "../data/bok"
cfg.save_dataframe.output_file = "bok_minutes_tokenized.parquet"

df = eKonf.instantiate(cfg)
df.tail()

Tokenizing column: text:   0%|          | 0/163 [00:00<?, ?it/s]

,id,text,sent_id
162,162,앞/NNG 으로/JKB /SP 코로나/NNG 19/SN /SP 충격/NNG 에서/J...,337
162,162,또한/MAJ /SP 글로벌/NNG /SP 공급/NNG 망/NNG /SP 재편/NNG...,338
162,162,향후/NNG /SP 경제/NNG 회복세/NNG 와/JC /SP 물가/NNG 의/JK...,339
162,162,,340
162,162,Government/SL '/SY s/SL /SP View/SL,341


In [4]:
tkn_cfg = eKonf.compose(config_group="preprocessor/tokenizer=mecab_econ")
tkn_cfg.extract.strip_pos = False

cfg = eKonf.compose(config_group="pipeline")
cfg.data.data_dir = "../data/bok"
cfg.data.data_file = "bok_minutes_tokenized.parquet"
cfg._pipeline_ = ["extract_tokens", "save_dataframe"]
cfg.num_workers = 100
cfg.verbose = True
cfg.extract_tokens.preprocessor.tokenizer = tkn_cfg
cfg.extract_tokens.nouns_only = False
cfg.save_dataframe.output_dir = "../data/bok"
cfg.save_dataframe.output_file = "bok_minutes_tokens.parquet"
df = eKonf.instantiate(cfg)
df.tail()

Extracting column: text:   0%|          | 0/33857 [00:00<?, ?it/s]

,id,text,sent_id
162,162,앞/NNG 으로/JKB 코로나/NNG 19/SN 충격/NNG 에서/JKB 점차/MA...,337
162,162,또한/MAJ 글로벌/NNG 공급/NNG 망/NNG 재편/NNG 기후/NNG 변화/N...,338
162,162,향후/NNG 경제/NNG 회복세/NNG 와/JC 물가/NNG 의/JKG 흐름/NNG...,339
162,162,,340
162,162,Government/SL s/SL View/SL,341


## Train ngrams

In [5]:
ngram_cfg = eKonf.compose(config_group="model/ngram=npmi")
ngram_cfg.data.data_dir = "../data/bok"
ngram_cfg.data.data_file = "bok_minutes_tokens.parquet"
ngram_cfg.verbose = True
ngram_cfg.autoload = True
ngram_cfg.candidates.threshold = 0.4
ngram = eKonf.instantiate(ngram_cfg)

In [6]:
print(len(ngram.ngrams), len(ngram.candidates))

0 41005


In [7]:
ngram.postag_rules = [["/NN", "/J", "/NN"], ["/XP", "/NN"], ["/XP", "/NN", "/NN"]]
# ngram.postag_rules = ['/NN']

ngram.export_ngrams(threshold=0.8, apply_postag_rules=True)

[(('민/XPN', '감도/NNG'),
  score(words='민/XPN;감도/NNG', score=0.9905600642226081, frequency=27, length=44827)),
 (('신/XPN', '흥국/NNG'),
  score(words='신/XPN;흥국/NNG', score=0.9602497345784399, frequency=999, length=44827)),
 (('문안/NNG', '을/JKO', '작성/NNG'),
  score(words='문안/NNG;을/JKO;작성/NNG', score=0.9550163996733796, frequency=43, length=44827)),
 (('예/NNG', '의/JKG', '주시/NNG'),
  score(words='예/NNG;의/JKG;주시/NNG', score=0.952158528049305, frequency=193, length=44827)),
 (('한국/NNP', '은/JX', '행법/NNG'),
  score(words='한국/NNP;은/JX;행법/NNG', score=0.9215496994789585, frequency=13, length=44827)),
 (('한/NNP', '은/JX', '법/NNG'),
  score(words='한/NNP;은/JX;법/NNG', score=0.9115135958462448, frequency=17, length=44827)),
 (('차관/NNG', '에게/JKB', '발언/NNG'),
  score(words='차관/NNG;에게/JKB;발언/NNG', score=0.8730837780556813, frequency=48, length=44827)),
 (('취지/NNG', '로/JKB', '발언/NNG'),
  score(words='취지/NNG;로/JKB;발언/NNG', score=0.8703677143686631, frequency=36, length=44827)),
 (('신/XPN', '용스/NNP', '프레드/NNP'),

In [8]:
ngram.load_data()
sentence = ngram.sentences[18]
print(sentence)
tokens = ngram.ngramize_sentence(sentence)
print(tokens)

이/NP 에/JKB 대해/VV+EC 관련/NNG 부서/NNG 에서/JKB 는/JX 실업/NNG 률/XSN 과/JC 경기/NNG 지수/NNG 와/JKB 의/JKG 상관/NNG 관계/NNG 는/JX 그렇게/MAG 높/VA 지/EC 않/VX 은/ETM 것/NNB 으로/JKB 나타나/VV 며/EC 통계청/NNG 에서/JKB 발표/NNG 하/XSV 는/ETM 공식/NNG 적/XSN 인/VCP+ETM 실업/NNG 률/XSN 을/JKO 구직/NNG 단념/NNG 자/XSN 를/JKO 포함/NNG 한/XSA+ETM 광의/NNG 의/JKG 실업/NNG 률/XSN 로/JKB 전환/NNG 할/XSV+ETM 경우/NNG 오히려/MAJ 경기/NNG 와/JKB 의/JKG 연관/NNG 성/XSN 이/JKS 좀/MAG 더/MAG 높/VA 은/ETM 것/NNB 으로/JKB 분석/NNG 되/XSV 므로/EC 경기/NNG 상환/NNG 판단/NNG 을/JKO 위해서/VV+EC 는/JX 공식/NNG 적/XSN 인/VCP+ETM 실업/NNG 률/XSN 보다/JKB 는/JX 광의/NNG 의/JKG 실업/NNG 률/XSN 을/JKO 이용/NNG 하/XSV 는/ETM 것/NNB 이/JKS 더/MAG 좋/VA 을/ETM 것/NNB 같/VA 다고/EC 설명/NNG 하/XSV 였/EP 음/ETN
['이', '에', '대해', '관련', '부서', '에서', '는', '실업', '률', '과', '경기', '지수', '와', '의', '상관', '관계', '는', '그렇게', '높', '지', '않', '은', '것', '으로', '나타나', '며', '통계청', '에서', '발표', '하', '는', '공식', '적', '인', '실업', '률', '을', '구직', '단념', '자', '를', '포함', '한', '광의의실업', '률', '로', '전환', '할', '경우', '오히려', '경기', '와', '의', '연관', '성', '이', '좀', '더', '높', '은', '것으로분석', '되', '므

In [9]:
tokens = ngram[sentence]
print(tokens)

['이', '에', '대해', '관련', '부서', '에서', '는', '실업', '률', '과', '경기', '지수', '와', '의', '상관', '관계', '는', '그렇게', '높', '지', '않', '은', '것', '으로', '나타나', '며', '통계청', '에서', '발표', '하', '는', '공식', '적', '인', '실업', '률', '을', '구직', '단념', '자', '를', '포함', '한', '광의의실업', '률', '로', '전환', '할', '경우', '오히려', '경기', '와', '의', '연관', '성', '이', '좀', '더', '높', '은', '것으로분석', '되', '므로', '경기', '상환', '판단', '을', '위해서', '는', '공식', '적', '인', '실업', '률', '보다', '는', '광의의실업', '률', '을', '이용', '하', '는', '것', '이', '더', '좋', '을', '것', '같', '다고', '설명', '하', '였', '음']


In [10]:
_ngrams =ngram.find_ngrams(
    ngram.sentences[10:105], strip_pos=False, surface_delim=";", threshold=0.5
)

100%|██████████| 95/95 [00:00<00:00, 229.54it/s]


In [11]:
_ngrams

{'의견/NNG;을/JKO;제시/NNG': {'score': 0.7712962057207168, 'count': 4},
 '것/NNB;으로/JKB;예상/NNG': {'score': 0.6434462999127033, 'count': 9},
 '광의/NNG;의/JKG;실업/NNG': {'score': 0.7125695666268386, 'count': 7},
 '위원/NNG;은/JX;최근/NNG': {'score': 0.6384959743711969, 'count': 6},
 '나라/NNG;의/JKG;경우/NNG': {'score': 0.5392882244391282, 'count': 1},
 '비중/NNG;을/JKO;차지/NNG': {'score': 0.7072775299321461, 'count': 2},
 '견해/NNG;를/JKO;피력/NNG': {'score': 0.7097555785919298, 'count': 4},
 '점/NNG;을/JKO;감안/NNG': {'score': 0.6377572632677098, 'count': 3},
 '견해/NNG;를/JKO;제시/NNG': {'score': 0.6669627043316719, 'count': 1},
 '것/NNB;으로/JKB;판단/NNG': {'score': 0.6210796296158267, 'count': 8},
 '요인/NNG;이/JKS;상존/NNG': {'score': 0.5211343183357702, 'count': 1},
 '부서/NNG;의/JKG;견해/NNG': {'score': 0.6754271644040637, 'count': 2},
 '요인/NNG;에/JKB;기인/NNG': {'score': 0.5043339218152425, 'count': 1},
 '예/NNG;의/JKG;주시/NNG': {'score': 0.952158528049305, 'count': 2},
 '것/NNB;으로/JKB;생각/NNG': {'score': 0.5958825911374661, 'count': 8},